# Reto 8 - Nifi + Kafka + Python Notebook + BD

## Script reto 5: mostrar por pantalla los mensajes del topic "simpsons"


In [28]:
!pip install kafka-python

     |████████████████████████████████| 246 kB 1.7 MB/s eta 0:00:01


In [60]:
from kafka import KafkaConsumer

topic = 'simpsons-quotes'
bootstrap_servers = 'docker_test-kafka-1:29092'
consumer = KafkaConsumer(topic, bootstrap_servers=bootstrap_servers, auto_offset_reset='earliest')

for msg in consumer:
    print(msg.value.decode("utf-8"))

{"quote":"And this is the snack holder where I can put my beverage or, if you will, cupcake.","character":"Homer Simpson","image":"https://cdn.glitch.com/3c3ffadc-3406-4440-bb95-d40ec8fcde72%2FHomerSimpson.png?1497567511939","characterDirection":"Right"}
      
{"quote":"I hope I didn't brain my damage.","character":"Homer Simpson","image":"https://cdn.glitch.com/3c3ffadc-3406-4440-bb95-d40ec8fcde72%2FHomerSimpson.png?1497567511939","characterDirection":"Right"}
      
{"quote":"Can't we have one meeting that doesn't end with us digging up a corpse?","character":"Mayor Quimby","image":"https://cdn.glitch.com/3c3ffadc-3406-4440-bb95-d40ec8fcde72%2FMayorQuimby.png?1497627527799","characterDirection":"Left"}
      
{"quote":"Last night's 'Itchy and Scratchy' was, without a doubt, the worst episode ever. Rest assured that I was on the Internet within minutes, registering my disgust throughout the world.","character":"Comic Book Guy","image":"https://cdn.glitch.com/3c3ffadc-3406-4440-bb95-d

KeyboardInterrupt: 

## Script reto 6: cargar datos mensajes en PostgreSQL

In [32]:
!pip install psycopg2-binary 

     |████████████████████████████████| 3.0 MB 1.5 MB/s eta 0:00:01


In [51]:
from kafka import KafkaConsumer
import json
import psycopg2

topic = 'simpsons-quotes'
bootstrap_servers = 'docker_test-kafka-1:29092'
consumer = KafkaConsumer(topic, bootstrap_servers=bootstrap_servers, auto_offset_reset='earliest')

conn = psycopg2.connect(
    host="docker_test-postgres-1",
    port="5432",
    database="simpsons",
    user="root",
    password="1234")

cursor = conn.cursor()


In [ ]:
# Insertar mensajes en PostgreSQL
insert1 = "INSERT INTO public.characters (name,image) VALUES(%s,%s);"
insert2 = "INSERT INTO public.quotes (quote,\"character\",\"characterDirection\") VALUES(%s,%s,%s);"

for msg in consumer:
    if (msg.offset % 2) == 0:
        data = json.loads(msg.value.decode("utf-8"))
        cursor.execute(insert1,(data['character'], data['image']))
        cursor.execute(insert2,(data['quote'],data['character'],data['characterDirection']))
        conn.commit()

## Script reto 7: cargar datos mensajes en MongoDB

In [35]:
!pip install pymongo

In [55]:
import pymongo
from kafka import KafkaConsumer
import json

topic = 'simpsons-quotes'
bootstrap_servers = 'docker_test-kafka-1:29092'
consumer = KafkaConsumer(topic, bootstrap_servers=bootstrap_servers, auto_offset_reset='earliest')

client=pymongo.MongoClient(host='mongo',port=27017,username='root',password='1234')

simpsons_db=client["simpsons"]

quotes_col=simpsons_db["quotes"]

In [ ]:
# Insertar mensajes en MongoDB
for msg in consumer:
    if (msg.offset % 2) == 0:
        quote = json.loads(msg.value.decode("utf-8"))
        quotes_col.insert_one(quote)

## Consultas a las bases de datos
### Consultas a PostgreSQL

- Frases dichas por el personaje "Homer Simpson":

In [52]:
query = "SELECT quote FROM public.quotes WHERE character='Homer Simpson'"
cursor.execute(query)
print(cursor.fetchall())

[('And this is the snack holder where I can put my beverage or, if you will, cupcake.',), ("I hope I didn't brain my damage.",), ("All I'm gonna use this bed for is sleeping, eating and maybe building a little fort.",), ("Facts are meaningless. You could use facts to prove anything that's even remotely true.",)]


- Imágenes de personajes cuyo nombre empieza por 'B':

In [53]:
query = "SELECT image FROM public.characters WHERE name LIKE 'B%'"
cursor.execute(query)
print(cursor.fetchall())

[('https://cdn.glitch.com/3c3ffadc-3406-4440-bb95-d40ec8fcde72%2FBartSimpson.png?1497567511638',)]


- Número de frases por personaje:

In [54]:
query = "SELECT count(*) AS num_quotes, character FROM public.quotes GROUP BY character"
cursor.execute(query)
print(cursor.fetchall())

[(2, 'Apu Nahasapeemapetilon'), (1, 'Bart Simpson'), (1, 'Ralph Wiggum'), (4, 'Homer Simpson'), (1, 'Mayor Quimby'), (1, 'Comic Book Guy')]


In [ ]:
cursor.close()

### Consultas a MongoDB

- Documentos con frase dicha por el personaje "Homer Simpson":

In [56]:
query = {"character": "Homer Simpson"}
docs = quotes_col.find(query)

for doc in docs:
    print(doc)

{'_id': ObjectId('633fe946f5f905faac34a965'), 'quote': 'And this is the snack holder where I can put my beverage or, if you will, cupcake.', 'character': 'Homer Simpson', 'image': 'https://cdn.glitch.com/3c3ffadc-3406-4440-bb95-d40ec8fcde72%2FHomerSimpson.png?1497567511939', 'characterDirection': 'Right'}
{'_id': ObjectId('633fe946f5f905faac34a966'), 'quote': "I hope I didn't brain my damage.", 'character': 'Homer Simpson', 'image': 'https://cdn.glitch.com/3c3ffadc-3406-4440-bb95-d40ec8fcde72%2FHomerSimpson.png?1497567511939', 'characterDirection': 'Right'}
{'_id': ObjectId('633fe946f5f905faac34a96a'), 'quote': "All I'm gonna use this bed for is sleeping, eating and maybe building a little fort.", 'character': 'Homer Simpson', 'image': 'https://cdn.glitch.com/3c3ffadc-3406-4440-bb95-d40ec8fcde72%2FHomerSimpson.png?1497567511939', 'characterDirection': 'Right'}
{'_id': ObjectId('633fe946f5f905faac34a96d'), 'quote': "Facts are meaningless. You could use facts to prove anything that's eve

- Documentos con frase que empiece con la letra "A":

In [59]:
query = {"quote": {"$regex": "^A"}}
docs = quotes_col.find(query)

for doc in docs:
    print(doc)

{'_id': ObjectId('633fe946f5f905faac34a965'), 'quote': 'And this is the snack holder where I can put my beverage or, if you will, cupcake.', 'character': 'Homer Simpson', 'image': 'https://cdn.glitch.com/3c3ffadc-3406-4440-bb95-d40ec8fcde72%2FHomerSimpson.png?1497567511939', 'characterDirection': 'Right'}
{'_id': ObjectId('633fe946f5f905faac34a96a'), 'quote': "All I'm gonna use this bed for is sleeping, eating and maybe building a little fort.", 'character': 'Homer Simpson', 'image': 'https://cdn.glitch.com/3c3ffadc-3406-4440-bb95-d40ec8fcde72%2FHomerSimpson.png?1497567511939', 'characterDirection': 'Right'}
